(target-dj-querying-data)=
# DataJoint pipeline: Querying data

:::{important}
Before getting started, ensure that you have a [DataJoint pipeline deployed](target-dj-pipeline-deployment) and that [data has been ingested into the pipeline](target-dj-data-ingestion-processing).
:::

This notebook provides examples of how to query data from the [Aeon DataJoint pipeline](target-aeon-dj-pipeline). 

:::{note}
The examples in this notebook use the [Single mouse in a foraging assay](sample-data-single-mouse-foraging:) dataset, specifically the experiment named `social0.2-aeon3`. 
If you are using a different dataset, be sure to replace the experiment name and  parameters in the code below accordingly.
:::

In [1]:
from aeon.dj_pipeline import acquisition, tracking
from aeon.dj_pipeline.analysis import block_analysis

[2025-02-06 17:08:32,281][INFO]: Connecting milagros@db.datajoint.com:3306
[2025-02-06 17:08:32,423][INFO]: Connected milagros@db.datajoint.com:3306


DataJoint offers various [query operators](datajoint:docs/core/datajoint-python/0.14/query/operators/), that allow for powerful data manipulations. These include restriction (&), projection, joining (*), and aggregation, enabling flexible data analysis workflows.

## Querying Acquisition Data

The acquisition module manages raw data collected during experiments. We'll start by exploring the `acquisition.Chunk` table, which stores metadata about discrete time chunks and the associated raw data files for each experiment.

In [2]:
acquisition.Chunk()

experiment_name e.g exp0-aeon3,chunk_start datetime of the start of a given acquisition chunk,chunk_end datetime of the end of a given acquisition chunk,directory_type,epoch_start
social0.2-aeon3,2024-03-02 12:00:00,2024-03-02 13:00:00,raw,2024-03-01 16:46:12
social0.2-aeon3,2024-03-02 13:00:00,2024-03-02 14:00:00,raw,2024-03-01 16:46:12


This command retrieves all records from the `Chunk` table. While comprehensive, it's often more efficient to filter the data using specific criteria.

### Restricting Data by Experiment

As an example, we can use the [restriction operator `&`](datajoint:docs/core/datajoint-python/0.14/query/operators/#restriction) to focus on data from a specific experiment. For that, we define a restriction key. 

In [3]:
experiment_key = {"experiment_name": "social0.2-aeon3"}
acquisition.Chunk & experiment_key


experiment_name e.g exp0-aeon3,chunk_start datetime of the start of a given acquisition chunk,chunk_end datetime of the end of a given acquisition chunk,directory_type,epoch_start
social0.2-aeon3,2024-03-02 12:00:00,2024-03-02 13:00:00,raw,2024-03-01 16:46:12
social0.2-aeon3,2024-03-02 13:00:00,2024-03-02 14:00:00,raw,2024-03-01 16:46:12


All {term}`chunks <Acquisition Chunk>` associated with the specific experiment `social0.2-aeon3` have been retrieved using the restriction operator `&`. 


### Querying Tracking Data

Tracking data provides detailed information on the movement and positioning of subjects. We'll apply the same experiment filter to explore different tracking-related tables.

Let's apply the same restriction by experiment to query position tracking data with [SLEAP](sleap:). The `SLEAPTracking` table contains the position tracking of object(s) from a particular `VideoSource` per chunk.

The same restriction key for the experiment `social0.2-aeon3` is applied:

In [4]:
tracking.SLEAPTracking & experiment_key

experiment_name e.g exp0-aeon3,chunk_start datetime of the start of a given acquisition chunk,device_serial_number,spinnaker_video_source_install_time time of the spinnaker_video_source placed and started operation at this position,tracking_paramset_id
social0.2-aeon3,2024-03-02 12:00:00,23032909,2024-03-01 16:46:12,1
social0.2-aeon3,2024-03-02 13:00:00,23032909,2024-03-01 16:46:12,1


The `PoseIdentity` table identifies each subject and records the body part used as an "anchor" for tracking purposes:


In [5]:
tracking.SLEAPTracking.PoseIdentity & experiment_key

experiment_name e.g exp0-aeon3,chunk_start datetime of the start of a given acquisition chunk,device_serial_number,spinnaker_video_source_install_time time of the spinnaker_video_source placed and started operation at this position,tracking_paramset_id,identity_idx,identity_name,identity_likelihood,anchor_part the name of the point used as anchor node for this class
social0.2-aeon3,2024-03-02 12:00:00,23032909,2024-03-01 16:46:12,1,0,BAA-1104045,=BLOB=,centroid
social0.2-aeon3,2024-03-02 12:00:00,23032909,2024-03-01 16:46:12,1,1,BAA-1104047,=BLOB=,centroid
social0.2-aeon3,2024-03-02 13:00:00,23032909,2024-03-01 16:46:12,1,0,BAA-1104045,=BLOB=,centroid
social0.2-aeon3,2024-03-02 13:00:00,23032909,2024-03-01 16:46:12,1,1,BAA-1104047,=BLOB=,centroid


Query the `Part` table to obtain x, y coordinates for all tracked body parts over time:

In [6]:
tracking.SLEAPTracking.Part & experiment_key

experiment_name e.g exp0-aeon3,chunk_start datetime of the start of a given acquisition chunk,device_serial_number,spinnaker_video_source_install_time time of the spinnaker_video_source placed and started operation at this position,tracking_paramset_id,identity_idx,part_name,sample_count number of data points acquired from this stream for a given chunk,x,y,likelihood,timestamps
social0.2-aeon3,2024-03-02 12:00:00,23032909,2024-03-01 16:46:12,1,0,centroid,19570,=BLOB=,=BLOB=,=BLOB=,=BLOB=
social0.2-aeon3,2024-03-02 12:00:00,23032909,2024-03-01 16:46:12,1,1,centroid,40429,=BLOB=,=BLOB=,=BLOB=,=BLOB=
social0.2-aeon3,2024-03-02 13:00:00,23032909,2024-03-01 16:46:12,1,0,centroid,19654,=BLOB=,=BLOB=,=BLOB=,=BLOB=
social0.2-aeon3,2024-03-02 13:00:00,23032909,2024-03-01 16:46:12,1,1,centroid,40342,=BLOB=,=BLOB=,=BLOB=,=BLOB=


### Querying Block-Level Data

`BlockAnalysis` contains block-level data that aggregates experimental events into defined time blocks. This allows for higher-level analyses, such as behavioral trends over extended periods.

Let's filter blocks using the same restriction key for experiment `social0.2-aeon3` and for blocks longer than 2 hours:

In [14]:
block_analysis.Block & experiment_key & 'block_duration_hr > 1'


experiment_name e.g exp0-aeon3,block_start,block_end,block_duration_hr (hour)
social0.2-aeon3,2024-03-02 12:09:41.013984,2024-03-02 14:00:00,1.839


In [15]:
block_analysis.BlockAnalysis & experiment_key & 'block_duration > 1'

experiment_name e.g exp0-aeon3,block_start,block_duration (hour),patch_count number of patches in the block,subject_count number of subjects in the block
social0.2-aeon3,2024-03-02 12:09:41.013984,1.83752,3,2


A `Block` refers to a specific period of time, typically lasting around 3 hours, during which the reward rate for each patch is predefined to facilitate certain animal behaviors. 

Let's choose a specific block for in-depth analysis:

In [16]:
block_analysis.Block & experiment_key & 'block_start LIKE "%2024-03-02%"'

experiment_name e.g exp0-aeon3,block_start,block_end,block_duration_hr (hour)
social0.2-aeon3,2024-03-02 12:09:41.013984,2024-03-02 14:00:00,1.839


To [`fetch()`](datajoint:docs/core/datajoint-python/0.14/query/fetch/#fetch) the [primary keys](datajoint:docs/core/datajoint-python/0.14/design/tables/primary/) of this specific _block_ in experiment `social0.2-aeon3`:

In [17]:
block_key = (block_analysis.Block & experiment_key & 'block_start LIKE "%2024-03-02%"').fetch("KEY")
block_key

[{'experiment_name': 'social0.2-aeon3',
  'block_start': datetime.datetime(2024, 3, 2, 12, 9, 41, 13984)}]

### Joining Data

The `BlockAnalysis` table aggregates data from multiple subjects and patches within a _block_: 

In [27]:
block_analysis.BlockAnalysis & block_key

experiment_name e.g exp0-aeon3,block_start,block_duration (hour),patch_count number of patches in the block,subject_count number of subjects in the block
social0.2-aeon3,2024-03-02 12:09:41.013984,1.83752,3,2


`BlockSubjectAnalysis` offers a detailed analysis focused on individual subjects, examining their interactions within a specific _block_:

In [19]:
block_analysis.BlockSubjectAnalysis & block_key

experiment_name e.g exp0-aeon3,block_start
social0.2-aeon3,2024-03-02 12:09:41.013984


Understanding subjects' patch `Preferences` based on time spent or distance traveled for this _block_:

In [20]:
block_analysis.BlockSubjectAnalysis.Preference & block_key

experiment_name e.g exp0-aeon3,block_start,"patch_name e.g. Patch1, Patch2",subject_name,cumulative_preference_by_wheel,cumulative_preference_by_time,running_preference_by_time,running_preference_by_wheel,final_preference_by_wheel cumulative_preference_by_wheel at the end of the block,final_preference_by_time cumulative_preference_by_time at the end of the block
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch1,BAA-1104045,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.0,0.0355948
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch1,BAA-1104047,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.105216,0.161178
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch2,BAA-1104045,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.235068,0.300882
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch2,BAA-1104047,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.352128,0.368936
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch3,BAA-1104045,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.764795,0.663523
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch3,BAA-1104047,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.542656,0.469887


Tracking the `Patch` interactions of each subject with different patches (areas of interests):


In [21]:
# subject position
block_analysis.BlockSubjectAnalysis.Patch & block_key


experiment_name e.g exp0-aeon3,block_start,"patch_name e.g. Patch1, Patch2",subject_name,in_patch_timestamps timestamps in which a particular subject is spending time at a particular patch,in_patch_time total seconds spent in this patch for this block,pellet_count,pellet_timestamps,patch_threshold patch threshold value at each pellet delivery,wheel_cumsum_distance_travelled wheel's cumulative distance travelled
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch1,BAA-1104045,=BLOB=,11.7,0,=BLOB=,=BLOB=,=BLOB=
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch1,BAA-1104047,=BLOB=,300.0,0,=BLOB=,=BLOB=,=BLOB=
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch2,BAA-1104045,=BLOB=,98.9,0,=BLOB=,=BLOB=,=BLOB=
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch2,BAA-1104047,=BLOB=,686.7,9,=BLOB=,=BLOB=,=BLOB=
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch3,BAA-1104045,=BLOB=,218.1,0,=BLOB=,=BLOB=,=BLOB=
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch3,BAA-1104047,=BLOB=,874.6,35,=BLOB=,=BLOB=,=BLOB=


By [joining (*)](datajoint:docs/core/datajoint-python/0.14/query/operators/#join-compatibility) the `Patch` and `Preference` part tables, data from both tables is consolidated, presenting a comprehensive view of patch-preference interactions.

In [28]:
block_analysis.BlockSubjectAnalysis.Patch * block_analysis.BlockSubjectAnalysis.Preference & block_key

experiment_name e.g exp0-aeon3,block_start,"patch_name e.g. Patch1, Patch2",subject_name,in_patch_timestamps timestamps in which a particular subject is spending time at a particular patch,in_patch_time total seconds spent in this patch for this block,pellet_count,pellet_timestamps,patch_threshold patch threshold value at each pellet delivery,wheel_cumsum_distance_travelled wheel's cumulative distance travelled,cumulative_preference_by_wheel,cumulative_preference_by_time,running_preference_by_time,running_preference_by_wheel,final_preference_by_wheel cumulative_preference_by_wheel at the end of the block,final_preference_by_time cumulative_preference_by_time at the end of the block
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch1,BAA-1104045,=BLOB=,11.7,0,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.0,0.0355948
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch1,BAA-1104047,=BLOB=,300.0,0,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.105216,0.161178
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch2,BAA-1104045,=BLOB=,98.9,0,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.235068,0.300882
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch2,BAA-1104047,=BLOB=,686.7,9,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.352128,0.368936
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch3,BAA-1104045,=BLOB=,218.1,0,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.764795,0.663523
social0.2-aeon3,2024-03-02 12:09:41.013984,Patch3,BAA-1104047,=BLOB=,874.6,35,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,0.542656,0.469887


### Fetching and Inspecting Patch Data

Finally, we can fetch detailed patch data for the selected _block_ to analyze it further, potentially using pandas for advanced data manipulations:

In [23]:
block_patch_data = (block_analysis.BlockAnalysis.Patch & block_key).fetch(format="frame").reset_index()

In [24]:
block_patch_data

,experiment_name,block_start,patch_name,pellet_count,pellet_timestamps,wheel_cumsum_distance_travelled,wheel_timestamps,patch_threshold,patch_threshold_timestamps,patch_rate,patch_offset
0,social0.2-aeon3,2024-03-02 12:09:41.013984,Patch1,0,[],"[-0.0, 0.006136297681430314, 0.006136297681430...","[2024-03-02T12:09:41.020000000, 2024-03-02T12:...",[],[],0.0100,75.0
1,social0.2-aeon3,2024-03-02 12:09:41.013984,Patch2,9,"[2024-03-02T12:24:33.777503967, 2024-03-02T12:...","[-0.0, -0.006136297681429426, -0.0076703721017...","[2024-03-02T12:09:41.020000000, 2024-03-02T12:...","[364.05544835561926, 164.93215066599433, 485.7...","[2024-03-02T12:24:09.392000198, 2024-03-02T12:...",0.0020,75.0
2,social0.2-aeon3,2024-03-02 12:09:41.013984,Patch3,35,"[2024-03-02T12:20:17.155488014, 2024-03-02T12:...","[-0.0, -0.0, 0.0076703721017885584, 0.00306814...","[2024-03-02T12:09:41.020000000, 2024-03-02T12:...","[153.60720025718558, 166.6737735450297, 327.20...","[2024-03-02T12:20:01.725984097, 2024-03-02T12:...",0.0033,75.0


This DataFrame structure makes it easy to perform additional analyses, visualizations, or statistical tests.